# Implied Volatility Surface Visualization

This notebook demonstrates how to:
1. Fetch orderbook data for options
2. Prepare features (mid price, spread, etc.)
3. Compute implied volatilities
4. Create an interactive IV surface plot with time slider


In [ ]:
%load_ext autoreload
%autoreload 2

from utils import *
from datetime import datetime, timedelta
import pandas as pd
import numpy as np


## 1. Fetch Orderbook Data

Fetch orderbook snapshots for BTC options. Module 6 = orderbook data.


In [ ]:
# Define date range (e.g., 1 day of data)
end_date = datetime(2024, 10, 20)
start_date = end_date - timedelta(days=1)

# Fetch orderbook data for BTC-USD options
# Using depth=5 to keep only top 5 levels and reduce memory usage
orderbook_df = fetch_market_data(
    module=6,
    inst_type='OPTION',
    inst_family_list='BTC-USD',
    start_date=start_date,
    end_date=end_date,
    date_aggr_type='daily',
    depth=5,
    verbose=True
)

print(f"\nFetched {len(orderbook_df):,} orderbook snapshots")
print(f"Columns: {len(orderbook_df.columns)}")
orderbook_df.head()


## 2. Prepare Features

Extract bid/ask/mid prices and calculate spreads. You'll need to provide a forward price for the underlying.


In [ ]:
# You can get the forward price from futures data or use spot price
# For this example, we'll use a fixed price (replace with actual data)
FORWARD_PRICE = 67000.0  # BTC forward price in USD

# Prepare features for all orderbook snapshots
features_df = prepare_orderbook_features(orderbook_df, FORWARD_PRICE)

print(f"Prepared {len(features_df):,} feature rows")
print(f"\nFeatures columns: {features_df.columns.tolist()}")
print(f"\nAverage spread: {features_df['spread'].mean():.2f}")
print(f"Average spread %: {features_df['spread_pct'].mean():.2f}%")
features_df.head()


## 3. Compute Implied Volatility Surface

Calculate IV for each option using the Black-76 model.


In [ ]:
# Compute IV surface (this may take a minute or two)
iv_df = compute_iv_surface(features_df, risk_free_rate=0.0)

print(f"Computed {len(iv_df):,} IV points")
print(f"Valid IVs: {iv_df['implied_vol'].notna().sum():,}")
print(f"\nIV Statistics:")
print(iv_df['implied_vol'].describe())

# Show a sample
iv_df.head(10)


## 4. Visualize IV Surface with Time Slider

Create an interactive 3D plot where you can scroll through time.


In [ ]:
# Create interactive IV surface plot
# time_step_minutes controls how often we take snapshots (5 min intervals)
fig = create_iv_surface_plot(
    iv_df,
    time_step_minutes=5,
    grid_resolution=40,
    title="BTC Options IV Surface"
)

fig.show()


## 5. Filter by Option Type

You can also visualize calls and puts separately.


In [ ]:
# Calls only
calls_df = iv_df[iv_df['option_type'] == 'C']
fig_calls = create_iv_surface_plot(
    calls_df,
    time_step_minutes=10,
    title="BTC Call Options IV Surface"
)
fig_calls.show()


In [ ]:
# Puts only
puts_df = iv_df[iv_df['option_type'] == 'P']
fig_puts = create_iv_surface_plot(
    puts_df,
    time_step_minutes=10,
    title="BTC Put Options IV Surface"
)
fig_puts.show()


## 6. Analyze Specific Time Slices

Look at the IV surface at specific points in time.


In [ ]:
# Convert timestamp to datetime for easier filtering
if iv_df['timestamp'].dtype in ['int64', 'float64']:
    iv_df['datetime'] = pd.to_datetime(iv_df['timestamp'], unit='ms')
else:
    iv_df['datetime'] = pd.to_datetime(iv_df['timestamp'])

# Pick a specific time
target_time = iv_df['datetime'].median()
window_minutes = 5

# Get data within window
time_slice = iv_df[
    (iv_df['datetime'] >= target_time - pd.Timedelta(minutes=window_minutes)) &
    (iv_df['datetime'] <= target_time + pd.Timedelta(minutes=window_minutes))
]

print(f"Time slice around {target_time}:")
print(f"  Points: {len(time_slice)}")
print(f"  Average IV: {time_slice['implied_vol'].mean():.2%}")
print(f"  Tenor range: {time_slice['tenor_years'].min():.2f} - {time_slice['tenor_years'].max():.2f} years")
print(f"  Strike range: ${time_slice['strike'].min():,.0f} - ${time_slice['strike'].max():,.0f}")
